# SuperCup Rowing Competition Analysis

This notebook analyzes crew progression through the SuperCup rowing competition stages using data from `VSc2x.xlsx`.

## Overview
- Import and process race data with slag information
- Track crew progression from Voorwedstrijden to Finales
- Visualize crew flows using Sankey diagrams
- Analyze patterns by veld categories

In [26]:
# Import required libraries and our custom modules
import pandas as pd
import numpy as np
import importlib

# Import our custom modules
import data_processing
import crew_tracking
import visualization

# Reload modules to get latest changes
importlib.reload(data_processing)
importlib.reload(crew_tracking)
importlib.reload(visualization)

from data_processing import load_and_process_excel
from crew_tracking import track_crew_progression_with_slag, analyze_crew_progression
from visualization import create_four_column_cumulative_sankey

print("All modules imported and reloaded successfully!")
print("Ready to create cumulative positioned Sankey diagrams!")

All modules imported and reloaded successfully!
Ready to create cumulative positioned Sankey diagrams!


In [27]:
# Load and process the Excel data
excel_file = "VSc2x.xlsx"
print(f"Loading data from {excel_file}...")

# Load the Excel file and split by race types
race_dataframes = load_and_process_excel(excel_file)

print(f"Successfully loaded {len(race_dataframes)} race types:")
for race_type, df in race_dataframes.items():
    print(f"- {race_type}: {len(df)} rows")
print()

Loading data from VSc2x.xlsx...
Successfully loaded 4 race types:
- voorwedstrijden: 38 rows
- challenges: 38 rows
- halve finales: 37 rows
- finales: 35 rows



In [28]:
# The race_dataframes already contain processed data split by race type
# We need to process the crew data to extract the proper structure for crew tracking
print("Processing race data for crew tracking...")

all_race_data = {}
all_crews = []

# The race_dataframes already have the data in the format we need
# We just need to convert them to the format expected by crew tracking
for race_type, df in race_dataframes.items():
    print(f"\nProcessing {race_type}:")
    
    # Convert dataframe rows to crew entries (excluding time columns)
    crew_entries = []
    for _, row in df.iterrows():
        crew_entry = {
            'pos': row.get('pos.', ''),
            'code': row.get('code', ''),
            'ploeg': row.get('ploeg', ''),
            'veld': row.get('veld', ''),
            'baan': row.get('baan', ''),
            'race_name': row.get('race_name', ''),
            'race_number': row.get('race_number', ''),
            'race_sub_number': row.get('race_sub_number', ''),
            'race_type': row.get('race_type', ''),
            'crew_member': row.get('crew_member', ''),
            'crew_unique_id': row.get('crew_unique_id', '')
        }
        crew_entries.append(crew_entry)
        all_crews.append(crew_entry)
    
    # Group by race_name for the structure expected by crew tracking
    race_data = {}
    for entry in crew_entries:
        race_name = entry['race_name']
        if race_name not in race_data:
            race_data[race_name] = []
        race_data[race_name].append(entry)
    
    all_race_data[race_type] = race_data
    print(f"  Found {len(race_data)} stages with total {len(crew_entries)} crew entries")

print(f"\nTotal crew entries across all races: {len(all_crews)}")

# Show example of data structure
if all_crews:
    print(f"\nExample crew entry:")
    example = all_crews[0]
    for key, value in example.items():
        print(f"  {key}: {value}")

Processing race data for crew tracking...

Processing voorwedstrijden:
  Found 5 stages with total 38 crew entries

Processing challenges:
  Found 7 stages with total 38 crew entries

Processing halve finales:
  Found 7 stages with total 37 crew entries

Processing finales:
  Found 6 stages with total 35 crew entries

Total crew entries across all races: 148

Example crew entry:
  pos: 1
  code: SKA
  ploeg: Skadi/Argo
  veld: LVG 2x
  baan: [2]
  race_name: za 10:20 - race 117 - [303] VSc 2x voorwedstrijd 1
  race_number: 117
  race_sub_number: 1
  race_type: voorwedstrijden
  crew_member: Bente Jonkman
  crew_unique_id: SKA_1_1


In [29]:
# Track crew progression through competition stages
print("Tracking crew progression through competition stages...")

crew_progressions = track_crew_progression_with_slag(all_race_data)

print(f"Successfully tracked {len(crew_progressions)} unique crews")

# Analyze the progression patterns
analysis_results = analyze_crew_progression(crew_progressions)

# Print detailed summary
print(crew_progressions)

Tracking crew progression through competition stages...
Successfully tracked 38 unique crews
{'SKA_Bente_Jonkman': {'code': 'SKA', 'ploeg': 'Skadi/Argo', 'veld': 'LVG', 'crew_member': 'Bente Jonkman', 'stages': {'Voorwedstrijden': 'Voorwedstrijden 1', 'Challenges': 'Challenges 5', 'Halve Finales': 'Halve Finales 2', 'Finales': 'Finales (B)'}}, 'TRI_Femke_Holleman': {'code': 'TRI', 'ploeg': 'Triton (Dev)', 'veld': 'VB', 'crew_member': 'Femke Holleman', 'stages': {'Voorwedstrijden': 'Voorwedstrijden 1', 'Challenges': 'Challenges 2', 'Halve Finales': 'Halve Finales 3', 'Finales': 'Finales (B)'}}, 'MAA_Ilse_Feenstra': {'code': 'MAA', 'ploeg': 'Maas', 'veld': 'VE', 'crew_member': 'Ilse Feenstra', 'stages': {'Voorwedstrijden': 'Voorwedstrijden 1', 'Challenges': 'Challenges 4', 'Halve Finales': 'Halve Finales 4', 'Finales': 'Finales (D)'}}, 'NER_Juliette_Potjer': {'code': 'NER', 'ploeg': 'Nereus (Dev)', 'veld': 'VG', 'crew_member': 'Juliette Potjer', 'stages': {'Voorwedstrijden': 'Voorwedstri

In [30]:
print("=== CREATING 4-COLUMN SANKEY DIAGRAM ===")
print("Showing complete flow: Veld → Halve Finale Groups → Individual Finales → Final Positions")
fig_four_column = create_four_column_cumulative_sankey(all_race_data)
fig_four_column.show()


=== CREATING 4-COLUMN SANKEY DIAGRAM ===
Showing complete flow: Veld → Halve Finale Groups → Individual Finales → Final Positions
Creating adaptive 4-column cumulative Sankey...
Detecting competition format...
Detected configuration:
  - Boat class: 2x
  - Veld categories: 5 found
  - Finale letters: ['A', 'B', 'C', 'D', 'E', 'F', 'G']
  - Halve finale groups: ['ABC', 'DEFG']
  - Prefix patterns: {'LG': 'LV', 'B': 'V', 'E': 'V', 'G': 'V'}
Processing halve finales data (veld → unlabeled halve finale nodes):
  Processing: zo 10:19 - race 218 - [303] VSc 2x halve-DEFG-finale 1 (5 crews)
  Processing: zo 10:23 - race 219 - [303] VSc 2x halve-DEFG-finale 2 (5 crews)
  Processing: zo 10:27 - race 220 - [303] VSc 2x halve-DEFG-finale 3 (5 crews)
  Processing: zo 10:31 - race 221 - [303] VSc 2x halve-DEFG-finale 4 (4 crews)
  Processing: zo 10:35 - race 222 - [303] VSc 2x halve-ABC-finale 1 (6 crews)
  Processing: zo 10:39 - race 223 - [303] VSc 2x halve-ABC-finale 2 (6 crews)
  Processing: zo